### 블랜딩 작업

- 첫 번째(배경 이미지)에서 ROI(관심 영역)을 선택하고 해당 영역의 값을 두번째 이미지의 값으로 대체한 다음 다시 추가해야함

- 그러나 두번째의 이미지에는 이미 흰 배경이 깔려있음

- 배경 제거(로고만 보이기 위해서)를 하기 위해 마스킹 절차와 비트 연산을 수행해야함

- 01.Blnded Image.ipynb 에서 했던 cv2.addWeighted 방식을 사용 할 수 없음(이미지를 합친 부분의 색이 달라짐)

#### 최종 목표(로고를 빨간색 글씨로 바꾼 뒤 자연스럽게 배경 이미지에 넣는 것) 

01. 배경이미지의 ROI를 찾고

02. 워터마크는 cv2.resize()후 cv2.cvtColor()를 통한 흑백화

03. 그 후, 워터마크는 cv2.threshold()를 통해 이미지 임계처리
    - (두 개의 결과를 반환하는데 첫 번째 결과인 ret은 스레시홀딩에 사용한 임계값이고, 두번째 결과인 out은 스레시홀딩이 적용된 바이너리 이미지입니다.)


04. 배경과 워터마크 이미지 둘다 cv2.bitwise_##를 통해 여러번(필요시!) 작업해줌

05. cv2.add()를 통해 두 이미지를 합쳐줌

In [30]:
import cv2
import matplotlib.image as mimg
import matplotlib.pyplot as plt

In [24]:
large_img = cv2.imread('ex_image.png')
watermark = cv2.imread('ex_image_logo.png')

In [25]:
x_offset = 400
y_offset = 170

In [35]:
# STEP 1
roi = large_img[y_offset:470, x_offset:700]

In [15]:
# STEP 2: logo image 빨간색 부분을 제외한 모든 것을 필터링 하도록 -> 회색조 이미지로 변경
small_img = cv2.resize(watermark, (300, 300))
small_img_gray = cv2.cvtColor(small_img, cv2.COLOR_BGR2GRAY)

In [16]:
# STEP 3: 임계처리(로고 값만 뽑아 비워둠) 
ret, mask = cv2.threshold(small_img_gray, 120, 255, cv2.THRESH_BINARY)


In [18]:
# STEP 4: cv2.bitwise_## 처리

# bg <= bitwise_or을 통해 배경과 로고만 빼고 비워있는 mask값을 OR을 통해 합침
bg = cv2.bitwise_or(roi, roi, mask=mask)

# mask_inv <= bitwise_not을 통해 배경이 비워진 로고 값만 추출
mask_inv = cv2.bitwise_not(mask)

In [19]:
# STEP 4: 원본 로고 이미지의 붉은 부분색을 로고색과 합침
# 결과적으로 배경이 비워진 붉은 로고색으로 변경
fg = cv2.bitwise_and(small_img, small_img, mask=mask_inv)

In [20]:
# STEP 5: 붉게 변한 로고(배경X)과 ROI영역(검은색 로고로 되어있는) 값을 합침
final_roi = cv2.add(bg, fg)

In [21]:
# 이미지에 수정된 FINAL_ROI를 붙여넣기
large_img[y_offset: y_offset + small_img.shape[0],
          x_offset: x_offset+small_img.shape[1]] = final_roi

In [22]:
cv2.imshow("test", large_img)
cv2.waitKey(0)

-1